In [1]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV


# Load data
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')


In [2]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])

X_train = tokenizer.texts_to_sequences(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_data['text'])

# Pad sequences
maxlen = 100  # adjust as needed
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Encode labels
label_mapping = {
    'bully-Spam': 0,
    'not_bully-Spam': 1,
    'bully-Ham': 2,
    'not_bully-Ham': 3
}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

In [3]:
from keras.layers import Bidirectional

# Build Bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Bidirectional LSTM model
history=model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1)


Epoch 1/15
40/40 [==============================] - 10s 124ms/step - loss: 1.1394 - accuracy: 0.4385 - val_loss: 0.7982 - val_accuracy: 0.8085
Epoch 2/15
40/40 [==============================] - 4s 88ms/step - loss: 0.8275 - accuracy: 0.5837 - val_loss: 0.9460 - val_accuracy: 0.3546
Epoch 3/15
40/40 [==============================] - 3s 72ms/step - loss: 0.6960 - accuracy: 0.7534 - val_loss: 0.5748 - val_accuracy: 0.9574
Epoch 4/15
40/40 [==============================] - 3s 72ms/step - loss: 0.4795 - accuracy: 0.8779 - val_loss: 0.8528 - val_accuracy: 0.6454
Epoch 5/15
40/40 [==============================] - 3s 73ms/step - loss: 0.2677 - accuracy: 0.9302 - val_loss: 0.7471 - val_accuracy: 0.7021
Epoch 6/15
40/40 [==============================] - 3s 71ms/step - loss: 0.1893 - accuracy: 0.9485 - val_loss: 0.7303 - val_accuracy: 0.7447
Epoch 7/15
40/40 [==============================] - 3s 73ms/step - loss: 0.1389 - accuracy: 0.9635 - val_loss: 0.6968 - val_accuracy: 0.7376
Epoch 8/15


In [4]:
# Evaluate Bi-LSTM model
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
y_pred = predictions.argmax(axis=1)
print("Bi-LSTM Classification Report:")
print(classification_report(y_test, y_pred))

14/14 [==============================] - 2s 27ms/step
Bi-LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        10
           1       0.89      0.81      0.85       115
           2       0.81      0.85      0.83       102
           3       0.84      0.84      0.84       199

    accuracy                           0.84       426
   macro avg       0.81      0.88      0.84       426
weighted avg       0.84      0.84      0.84       426



In [2]:
from keras.models import load_model
saved_model = load_model("final_bilstm.h5")

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
label_mapping = {
    'bully-Spam': 0,
    'not_bully-Spam': 1,
    'bully-Ham': 2,
    'not_bully-Ham': 3
}

In [11]:
import numpy as np

# Preprocess new text
new_text = "@harryc0422 thatâ€™s not true at all. I am a Chinese and most urban places in Zhejiang and Jiangsu have better infrastructure than US. I feel my life quality really sucks in the USâ€¦and I come from a small place within Ningbo, Zhejiang"
new_text_sequence = tokenizer.texts_to_sequences([new_text])
new_text_padded = pad_sequences(new_text_sequence, padding='post', maxlen=100)

# Make predictions on new text
new_text_prediction = saved_model.predict(new_text_padded)
predicted_label = np.argmax(new_text_prediction)

# Map predicted label back to original label
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_label_text = reverse_label_mapping[predicted_label]

print("Predicted label for the new text:", predicted_label_text)


1/1 [==============================] - 0s 47ms/step
Predicted label for the new text: not_bully-Ham


In [2]:
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')

In [3]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])

X_train = tokenizer.texts_to_sequences(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_data['text'])

# Pad sequences
maxlen = 100  # adjust as needed
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Encode labels
label_mapping = {
    'bully-Spam': 0,
    'not_bully-Spam': 1,
    'bully-Ham': 2,
    'not_bully-Ham': 3
}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

In [4]:
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam
import pandas as pd

# Define a list of optimizers to try along with their learning rates
optimizers = {
    'rmsprop': RMSprop(lr=0.001),
    'adadelta': Adadelta(lr=1.0),
    'nadam': Nadam(lr=0.002)
}


# Loop over each optimizer
for optimizer_name in optimizers:
    # Build DNN model
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=maxlen))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer=optimizers[optimizer_name], loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=0)
    
    # Evaluate on the test set
    predictions = model.predict(X_test)
    y_pred = predictions.argmax(axis=1)
    
    # Generate classification report
    print("Report for",optimizer_name)
    print(classification_report(y_test, y_pred))

C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adadelta.py:79: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\nitin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\nadam.py:86: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


14/14 [==============================] - 2s 25ms/step
Report for rmsprop
              precision    recall  f1-score   support

           0       1.00      0.30      0.46        10
           1       0.91      0.83      0.87       115
           2       0.80      0.85      0.82       102
           3       0.83      0.86      0.85       199

    accuracy                           0.84       426
   macro avg       0.88      0.71      0.75       426
weighted avg       0.85      0.84      0.84       426

14/14 [==============================] - 2s 25ms/step
Report for adadelta
              precision    recall  f1-score   support

           0       1.00      0.20      0.33        10
           1       0.91      0.78      0.84       115
           2       0.73      0.81      0.77       102
           3       0.82      0.87      0.85       199

    accuracy                           0.82       426
   macro avg       0.87      0.67      0.70       426
weighted avg       0.83      0.82     